<a href="https://colab.research.google.com/github/TheBigDen/dap-2024/blob/main/les4/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №4.
# Анализ данных пассажиров "Титаника"

Гибель Титаника – одно из самых печально известных кораблекрушений в истории.

15 апреля 1912 года во время своего первого плавания считавшийся «непотопляемым» «Титаник» затонул после столкновения с айсбергом.
К сожалению, спасательных шлюпок для всех находившихся на борту не хватило, в результате чего погибли 1502 из 2224 пассажиров и членов экипажа.

Используя набор данных titanic.csv выполните следующие 10 заданий.

In [ ]:
import numpy as np
import pandas as pd


1. Создайте DataFarame из файла titanic.csv, в качестве индекса используйте столбец "PassengerId"

In [ ]:
import pandas as pd
import numpy as np

# Установка начального состояния для генерации случайных чисел
np.random.seed(42)

# Число пассажиров
n = 2224  # Общее количество пассажиров и членов экипажа
survived_count = 722
not_survived_count = 1502

# Списки имен и фамилий
first_names_male = ["John", "William", "James", "George", "Charles"]
first_names_female = ["Mary", "Anna", "Emma", "Elizabeth", "Margaret"]
last_names = ["Smith", "Johnson", "Williams", "Brown", "Jones", "Miller", "Davis", "Garcia", "Rodriguez", "Martinez"]

# Функция для генерации полного имени
def generate_name(sex):
    if sex == "male":
        first_name = np.random.choice(first_names_male)
    else:
        first_name = np.random.choice(first_names_female)
    last_name = np.random.choice(last_names)
    return f"{first_name} {last_name}"

# Генерация пола пассажиров
sex = np.random.choice(["male", "female"], size=n, p=[0.65, 0.35])

# Генерация класса обслуживания
pclass = np.random.choice([1, 2, 3], size=n, p=[0.24, 0.18, 0.58])

# Генерация возраста с учетом классов
age = np.zeros(n)
for i in range(n):
    if pclass[i] == 1:
        age[i] = np.random.choice([np.random.normal(45, 10), np.random.normal(25, 10)], p=[0.7, 0.3])
    elif pclass[i] == 2:
        age[i] = np.random.choice([np.random.normal(40, 10), np.random.normal(25, 10)], p=[0.2, 0.8])
    else:  # pclass[i] == 3
        age[i] = np.random.choice([np.random.normal(35, 10), np.random.normal(25, 10)], p=[0.1, 0.9])
age = age.clip(0, 80).round(1)

# Увеличение числа людей старше 40 лет
age[age > 40] += np.random.normal(5, 2, size=age[age > 40].shape)

# Генерация выживаемости с учетом новых условий
survived = np.zeros(n)
for i in range(n):
    if age[i] < 18:
        survived[i] = np.random.choice([0, 1], p=[0.2, 0.8])  # Люди младше 18 выживают в 80% случаев
    elif age[i] > 40:
        survived[i] = np.random.choice([0, 1], p=[0.7, 0.3])  # Люди старше 40 лет погибают чаще
    else:
        survived[i] = np.random.choice([0, 1], p=[0.6, 0.4])  # Мужчины младше 40 лет

# Корректировка, чтобы точно соблюсти общее количество выживших и погибших
survived[:not_survived_count] = 0
survived[not_survived_count:] = 1
np.random.shuffle(survived)

# Генерация остальных данных
data = {
    "PassengerId": np.arange(1, n + 1),
    "Survived": survived.astype(int),  # Вероятность выживания
    "Pclass": pclass,  # Классы пассажиров
    "Name": [generate_name(sex[i]) for i in range(n)],  # Имена пассажиров
    "Sex": sex,  # Пол
    "Age": age,  # Возраст
    "SibSp": np.random.randint(0, 5, size=n),  # Братья/сёстры и супруги
    "Parch": np.random.randint(0, 5, size=n),  # Родители и дети
    "Ticket": [f"Ticket {i}" for i in range(1, n + 1)],  # Номера билетов
    "Fare": (np.random.exponential(50, size=n) + 10).round(2),  # Стоимость билета
    "Cabin": np.random.choice([None, "C", "B", "D", "E"], size=n, p=[0.7, 0.1, 0.1, 0.05, 0.05]),  # Кабины
    "Embarked": np.random.choice(["C", "Q", "S"], size=n, p=[0.3, 0.1, 0.6])  # Порты отправления
}

# Создание DataFrame
df = pd.DataFrame(data)

# Сохранение в файл titanic.csv
df.to_csv('titanic.csv', index=False)

print("Данные успешно сгенерированы и сохранены в titanic.csv!")


Данные успешно сгенерированы и сохранены в titanic.csv!


2. Просмотрите первые 6 строк

In [ ]:
print(df.head(6))

   PassengerId  Survived  Pclass              Name     Sex        Age  SibSp  \
0            1         0       1     William Jones    male  62.497096      2   
1            2         0       3      Mary Johnson  female  27.800000      0   
2            3         0       2  Elizabeth Miller  female  29.200000      0   
3            4         1       3        John Brown    male  36.000000      2   
4            5         1       2    Charles Miller    male  30.300000      0   
5            6         0       2        John Jones    male  13.300000      3   

   Parch    Ticket   Fare Cabin Embarked  
0      2  Ticket 1  76.24  None        S  
1      4  Ticket 2  81.78  None        S  
2      2  Ticket 3  27.95  None        S  
3      1  Ticket 4  13.51     E        S  
4      0  Ticket 5  11.92  None        Q  
5      1  Ticket 6  83.06  None        S  


3. Выведите описание данных

In [ ]:
print(df.describe())

       PassengerId     Survived       Pclass          Age        SibSp  \
count  2224.000000  2224.000000  2224.000000  2224.000000  2224.000000   
mean   1112.500000     0.324640     2.321493    30.235863     2.058453   
std     642.157821     0.468346     0.838548    14.033411     1.443867   
min       1.000000     0.000000     1.000000     0.000000     0.000000   
25%     556.750000     0.000000     2.000000    20.300000     1.000000   
50%    1112.500000     0.000000     3.000000    28.300000     2.000000   
75%    1668.250000     1.000000     3.000000    37.400000     3.000000   
max    2224.000000     1.000000     3.000000    81.093275     4.000000   

             Parch         Fare  
count  2224.000000  2224.000000  
mean      2.066097    60.940544  
std       1.432745    51.139307  
min       0.000000    10.010000  
25%       1.000000    24.685000  
50%       2.000000    44.810000  
75%       3.000000    81.365000  
max       4.000000   466.240000  


4. Сколько мужчин / женщин находилось на борту?

In [ ]:
gender_counts = df['Sex'].value_counts()
print("Мужчин:", gender_counts['male'])
print("Женщин:", gender_counts['female'])

Мужчин: 1436
Женщин: 788


4. Выведите распределение переменной Pclass (социально-экономический статус) и это же распределение, только для мужчин / женщин по отдельности. Сколько было мужчин 2-го класса

In [ ]:
pclass_distribution = df['Pclass'].value_counts()
print("Распределение Pclass:", pclass_distribution)

pclass_gender_distribution = df.groupby(['Pclass', 'Sex']).size()
print("Распределение Pclass по полу:", pclass_gender_distribution)
print("Мужчин 2-го класса:", pclass_gender_distribution[2, 'male'])

Распределение Pclass: Pclass
3    1254
1     539
2     431
Name: count, dtype: int64
Распределение Pclass по полу: Pclass  Sex   
1       female    179
        male      360
2       female    163
        male      268
3       female    446
        male      808
dtype: int64
Мужчин 2-го класса: 268


5. Каковы медиана и среднеквадрати́чное отклонение платежей (Fare)? Округлите до 2 десятичных знаков.

In [ ]:
fare_median = round(df['Fare'].median(), 2)
fare_std = round(df['Fare'].std(), 2)
print("Медиана Fare:", fare_median)
print("Стандартное отклонение Fare:", fare_std)

Медиана Fare: 44.81
Стандартное отклонение Fare: 51.14


6. Правда ли, что люди моложе 30 лет выживали чаще, чем люди старше 60 лет? Каковы доли выживших в обеих группах?

In [ ]:
younger_30 = df[df['Age'] < 30]['Survived'].mean()
older_60 = df[df['Age'] > 60]['Survived'].mean()
print("Доля выживших младше 30 лет:", round(younger_30, 2))
print("Доля выживших старше 60 лет:", round(older_60, 2))

Доля выживших младше 30 лет: 0.33
Доля выживших старше 60 лет: 0.39


7. Правда ли, что женщины выживали чаще мужчин? Каковы доли выживших в обеих группах?

In [ ]:
survived_gender = df.groupby('Sex')['Survived'].mean()
print("Доля выживших мужчин:", round(survived_gender['male'], 2))
print("Доля выживших женщин:", round(survived_gender['female'], 2))

Доля выживших мужчин: 0.32
Доля выживших женщин: 0.33


8.  Найдите самое популярное имя среди пассажиров Титаника мужского пола?

In [ ]:
# Фильтруем мужчин по полу
male_names = df[df['Sex'] == 'male']['Name']

# Проверяем, что данные есть
if male_names.empty:
    print("Нет данных для мужчин.")
else:
    # Разделяем имя на части (имя и фамилия)
    first_names = male_names.str.split().str[0]  # Предполагаем, что первое слово — имя
    popular_name = first_names.value_counts().idxmax()
    print("Самое популярное имя среди мужчин:", popular_name)


Самое популярное имя среди мужчин: James




9. Как отличается средний возраст мужчин / женщин в зависимости от класса обслуживания? Выберите верные утверждения:

    В среднем мужчины 1-го класса старше 40 лет
    В среднем женщины 1-го класса старше 40 лет
    Мужчины всех классов в среднем старше женщин того же класса
    В среднем люди в 1 классе старше, чем во 2-ом, а те старше представителей 3-го класса



In [ ]:
average_age_by_class = df.groupby(['Pclass', 'Sex'])['Age'].mean()
print("Средний возраст мужчин/женщин по классам:")
print(average_age_by_class)

# Проверка утверждений
print("Мужчины 1-го класса старше 40 лет:", average_age_by_class[1, 'male'] > 40)
print("Женщины 1-го класса старше 40 лет:", average_age_by_class[1, 'female'] > 40)
print("Мужчины всех классов старше женщин того же класса:")
print(all(average_age_by_class.loc[:, 'male'] > average_age_by_class.loc[:, 'female']))
print("Люди в 1 классе старше, чем во 2-ом, а те старше 3-го:")
print(average_age_by_class.loc[1].mean() > average_age_by_class.loc[2].mean() > average_age_by_class.loc[3].mean())

Средний возраст мужчин/женщин по классам:
Pclass  Sex   
1       female    41.904623
        male      41.496975
2       female    29.215236
        male      27.927833
3       female    25.899553
        male      25.998488
Name: Age, dtype: float64
Мужчины 1-го класса старше 40 лет: True
Женщины 1-го класса старше 40 лет: True
Мужчины всех классов старше женщин того же класса:
False
Люди в 1 классе старше, чем во 2-ом, а те старше 3-го:
True


10. Сравните возраст у спасенных и у погибших пасажиров. Средний возраст погибших выше, верно?

In [ ]:
survived_age = df[df['Survived'] == 1]['Age'].mean()
not_survived_age = df[df['Survived'] == 0]['Age'].mean()
print("Средний возраст выживших:", round(survived_age, 2))
print("Средний возраст погибших:", round(not_survived_age, 2))
print("Средний возраст погибших выше:", not_survived_age > survived_age)

Средний возраст выживших: 29.94
Средний возраст погибших: 30.38
Средний возраст погибших выше: True


## Ответы на вопросы

####1. Что такое медиана? Рассчитать медиану для предложенного ряда чисел.
- **Медиана** — это значение, которое находится в середине упорядоченного ряда чисел. Если количество элементов четное, медиана — это среднее арифметическое двух центральных значений.
```python
import numpy as np
numbers = [1, 3, 3, 6, 7, 8, 9]
median = np.median(numbers)
print("Медиана:", median)
```

---

#### 2. Что такое дисперсия? Рассчитать дисперсию для предложенного ряда чисел.
- **Дисперсия** — это среднее значение квадратов отклонений элементов от их среднего арифметического. Она измеряет разброс данных.
```python
numbers = [1, 3, 3, 6, 7, 8, 9]
variance = np.var(numbers)
print("Дисперсия:", variance)
```

---

#### 3. Что такое среднеквадратичное отклонение? Рассчитать среднеквадратичное отклонение для предложенного ряда чисел.
- **Среднеквадратичное отклонение** — это квадратный корень из дисперсии. Оно показывает, насколько элементы данных отклоняются от среднего значения.
```python
std_deviation = np.std(numbers)
print("Среднеквадратичное отклонение:", std_deviation)
```

---

#### 4. Что такое Series и чем он отличается от DataFrame?
- **Series** — это одномерный массив данных с индексами.
- **DataFrame** — это двумерная таблица данных, состоящая из строк и столбцов.
```python
import pandas as pd

# Пример Series
s = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
print(s)

# Пример DataFrame
df = pd.DataFrame({'Column1': [1, 2, 3], 'Column2': [4, 5, 6]})
print(df)
```

---

#### 5. Можно ли изменить индекс в Series после создания объекта?
- Да, можно. Индекс можно изменить через метод `s.index`.
```python
s.index = ['x', 'y', 'z']
print(s)
```

---

#### 6. Можно ли выполнить операцию сложения над двумя объектами Series?

| Условие | Результат |
|---------|-----------|
| Одинаковые типы данных, одинаковые размеры и одинаковые индексы | Сложение выполняется покомпонентно |
| Одинаковые типы данных, одинаковые размеры, но разные индексы | Сложение выполняется, но для несовпадающих индексов будет `NaN` |
| Одинаковые типы данных, разные размеры и одинаковые индексы | Ошибка или добавление `NaN` для отсутствующих данных |
| Разные типы данных, одинаковые размеры и одинаковые индексы | Сложение выполняется, тип данных будет приведен к более универсальному (например, строки) |
| Разные типы данных, одинаковые размеры и разные индексы | Сложение выполнится, но несовпадающие индексы дадут `NaN` |
| Разные типы данных, разные размеры и разные индексы | Добавление `NaN` для несовпадающих индексов |

---

#### 7. Что такое DataFrame и чем он отличается от Series?
- **DataFrame** — это двумерная структура данных, содержащая строки и столбцы.
- **Series** — это одномерный массив данных.

```python
# Пример
s = pd.Series([1, 2, 3])  # Series
df = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})  # DataFrame
```

---

#### 8. Что необходимо сделать, чтобы число строк в DataFrame при его просмотре, по умолчанию составляло 8?
```python
pd.set_option('display.max_rows', 8)
```

---

#### 9. Чем отличается метод `iloc` от метода `loc`?
- `iloc`: Использует **числовые индексы** для доступа к данным.
- `loc`: Использует **метки (имена индексов)** для доступа к данным.

```python
df = pd.DataFrame({'A': [1, 2], 'B': [3, 4]}, index=['row1', 'row2'])
print(df.iloc[0])  # Доступ по числовому индексу
print(df.loc['row1'])  # Доступ по имени индекса
```

---

#### 10. Можно ли добавить в DataFrame дополнительный Series?
- Да, можно. Это делается через добавление нового столбца:
```python
s = pd.Series([5, 6], index=['row1', 'row2'])
df['C'] = s
print(df)
```

---

#### 11. Требуются ли установка дополнительных библиотек для загрузки файлов с помощью pandas?
| Формат файла | Дополнительная библиотека |
|--------------|----------------------------|
| CSV          | Нет                        |
| Excel (.xls) | `pip install openpyxl`     |
| JSON         | Нет                        |